In [2]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:

from audio.redis import *
from audio.audio import *
#from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json
import time

In [5]:
async def get_next_chunk_start(redis_inner_client, audio_name, length,shift):
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    diarization_result = diarization.data

    if len(diarization_result)>0:
        last_speech = diarization_result[-1]

        ended_silence = length - last_speech['end']
        print(ended_silence)
        if ended_silence<1:
            print('interrupted')
            return last_speech['start']+shift
        

        else:
            print('non-interrupted') 
            return last_speech['end']+shift



    else: return None

In [6]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'
import subprocess
from pydub import AudioSegment
import io
from time import sleep

from audio.redis import Transcript

In [7]:
redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [8]:
async def diarize(client_id, audio_name, shift):
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    df = pd.DataFrame(diarization.data)
    df['len'] = df['end'] - df['start']

    if len(df)>0:
        expanded_df = pd.DataFrame(columns=['speaker', 'time'])
        for index, row in df.iterrows():
            time_range = np.arange(row['start'] * 100, row['end'] * 100, 1).astype(int)

            temp_df = pd.DataFrame({
                'speaker': row['speaker'],
                'time': time_range
            })
            expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
        expanded_df['time'] = (expanded_df['time'] / 100.0).astype('float')
        expanded_df['conv_time'] = (expanded_df['time'] + shift).astype('float')

        return expanded_df.sort_values("time")

In [9]:

async def transcribe(audio_name):
    await redis_inner_client.lpush('Audio2TranscribeQueue', f'{audio_name}:{client_id}')
    _,done = await redis_inner_client.brpop(f'TranscribeReady:{audio_name}')
    transcription =  Transcript(audio_name,redis_inner_client)
    await transcription.get()
    df =  pd.concat([pd.DataFrame(t) for t in transcription.data])


    df['start'] = df['start'].astype('float')
    return df.sort_values("start")

In [41]:
await get_connections_df('initialFeed_audio',redis_stream_client)

,queue,length


In [39]:
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]


connections =  await get_connections('initialFeed_audio',redis_stream_client)

In [40]:
[await redis_stream_client.delete(c) for c in connections]

[1]

In [14]:
connection_id = 'david_audio'

In [15]:

path = f'/app/testdata/{connection_id}.webm'

In [16]:
on_start = False

In [17]:

#on_start = True

In [18]:
start = 0
step = 30
max_length=60

In [20]:
start

58.63327674023769

In [30]:
redis_stream_client = await get_stream_redis()
redis_inner_client = await get_inner_redis()
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]
start = await redis_inner_client.rpop(f'Start:{connection_id}')
start = float(start) if start else 0
print('started at ',start)

#connection_id??
await writestream2file(connection_id,redis_stream_client)
print(start)
if on_start:  audio_slicer = await AudioSlicer.from_file(path,format = 'webm')
else       :  audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,max_length)

slice_duration = audio_slicer.audio.duration_seconds
print(slice_duration)

if slice_duration > step:

    audio_data = await audio_slicer.export_data()
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()

diarization_result, transcription_result = await asyncio.gather(
        diarize(client_id, audio_name, start),
        transcribe(audio_name)
    )

df = pd.merge_asof(transcription_result,diarization_result,left_on = 'start',right_on='time',direction='nearest')
df['speaker_change'] = df['speaker'] != df['speaker'].shift()
df['silence'] = df['start']-df['end'].shift()
df['speaker_change'] = np.where(df['silence']>2,True,df['speaker_change'])
df['speaker_change'] = df['speaker_change'].cumsum()
df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'}).join(df.groupby('speaker_change').apply(lambda x:''.join(x['word'])).to_frame('text'))
df['len'] = df['end'] - df['start']
df['speaker'] = np.where(df['len']>0.5,df['speaker'],np.nan)
df['speaker'] = df['speaker'].fillna(method='ffill').fillna(method='bfill')
df['conv_start'] = df['start']+start
df['conv_end'] = df['end']+start

start_ = await get_next_chunk_start(redis_inner_client, audio_name, slice_duration,start)
start = start_ if start_ else start+slice_duration
await redis_inner_client.lpush(f'Start:{connection_id}', start)


await redis_inner_client.lpush(f'Segment:{connection_id}', json.dumps(df.to_dict(orient='records')))

started at  562.0543293718167
562.0543293718167
60.048
None
0.3620916808149417
interrupted


/tmp/ipykernel_262942/1198295832.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'}).join(df.groupby('speaker_change').apply(lambda x:''.join(x['word'])).to_frame('text'))
/tmp/ipykernel_262942/1198295832.py:38: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['speaker'] = df['speaker'].fillna(method='ffill').fillna(method='bfill')


27

In [157]:
df

,speaker,start,end,text,len,conv_start,conv_end
speaker_change,,,,,,,
1,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,0.00,13.32,what concept of raid is the kind of framework...,13.32,219.388795,232.708795
2,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,14.04,14.32,would,0.28,233.428795,233.708795
3,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,14.32,19.88,be risks actions we've got above issues,5.56,233.708795,239.268795
4,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,22.54,28.78,and and this sometimes it's a double d so the...,6.24,241.928795,248.168795
5,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,32.33,36.33,the reason i mentioned all these things is th...,4.00,251.718795,255.718795
6,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,40.59,61.07,we can get if if your tool could break this s...,20.48,259.978795,280.458795
7,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,64.17,92.57,if you could then take those into whatever ma...,28.40,283.558795,311.958795
8,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,94.65,130.33,come across azure devops no no okay so ado wi...,35.68,314.038795,349.718795
9,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,133.56,142.96,uh it can then raise all these risks issues u...,9.40,352.948795,362.348795


In [158]:
diarization_result['speaker'].unique()

array(['dcde8443-9ac1-4a61-a1b6-7cd22dc9985b',
       '577fe873-b083-49ab-bde6-581670a11dde',
       '80d16e07-004c-4410-86fc-b2b7c4f54eb7'], dtype=object)

In [159]:
diarization_result.groupby('speaker').count()

,time,conv_time
speaker,,
577fe873-b083-49ab-bde6-581670a11dde,435,435
80d16e07-004c-4410-86fc-b2b7c4f54eb7,7357,7357
dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,35697,35697
